In [5]:
!pip install -U sentence-transformers

  Using cached sentence_transformers-2.7.0-py3-none-any.whl (171 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64

In [6]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
import ast

model = SentenceTransformer("sentence-transformers/LaBSE")

df = pd.read_csv('embeddings.csv')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [7]:
def find_similars(line, topic):
  embeddings = df.loc[df['topic'] == topic]['embeddings'].apply(ast.literal_eval).apply(np.array).tolist()
  embed_list = []
  for i in range(len(embeddings)):
    embed_list.append(embeddings[i])
  embeddings = np.array(embed_list)
  sentences = df.loc[df['topic'] == topic]['sentences'].tolist()

  # считаем по предложениям
  line_list = line.split('. ')
  # делаем список векторов для всех предложений в тексте новости
  line_list = [model.encode(line) for line in line_list]
  # и находим среднее
  line_embedding = sum(line_list)/len(line_list)

  # преобразуем в float64, чтобы нормально считало
  line_embedding = np.float64(line_embedding)
  # считаем косинусную близость между текстом и всеми тегами
  cos_sim = util.cos_sim(line_embedding, embeddings).tolist()
  cos_sim = sum(cos_sim, [])

  # находим список всех
  all_sentence_combinations = []
  for i in range(len(cos_sim)):
      all_sentence_combinations.append([cos_sim[i], sentences[i]])

  # сортируем по убыванию косинусной близости
  all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)
  seen = set()
  uniq = []

  for score, sentence in all_sentence_combinations:
      if sentence not in seen:
          uniq.append((score, sentence))
          seen.add(sentence)

  similars = []
  for score, sentence in uniq:
      similars.append((sentence, score))

  # находим лучшую косинусную близость
  best_score = similars[0][1]
  # устанавливаем процент отклонения от лучшей косинусной близости, который считаем приемлемым
  k = 0.95
  comparing_score = best_score * k
  best = []

  # находим лучшие
  for pair in similars:
    if pair[1] >= comparing_score:
      best.append(pair)

  return best

In [8]:
line = 'О чем фильм «Лед 3» Опытный хоккеист, вдовец, отец-одиночка Александр Горин работает тренером иркутской хоккейной команды. Крупные успехи им не светят, но у сибиряков есть собственная гордость, и когда в состав покупают молодую хоккейную звезду из Москвы, парни встречают талантливого форварда в штыки. А Сергей не находит ничего лучше, как заявить, что готов честно отыграть сезон, а потом свалить в НХЛ и забыть Иркутск как страшный сон. В воспитательных целях Горин вывозит команду потренироваться на байкальский лед. Сергей красиво забивает, но слышит треск льда. Иркутяне советуют ему лежать, не двигаясь, чтобы не провалиться… и сваливают. И Сергей лежит, пока вокруг него не начинает нарезать круги симпатичная фигуристка. Надя готовится к важным соревнованиям, но никак не может выполнить четверной тулуп. Отец против ее спортивной карьеры, так как боится, что девушку может постигнуть участь рано ушедшей матери. А Сергей пока не догадывается, что Надя — дочь его тренера. Зачем смотреть «Лед 3» — тот случай, когда спортивная музыкальная мелодрама, построенная на традиционных слезовыжимательных приемах, работает на все сто. Штампы? Да и наплевать, если эмоции во время просмотра зашкаливают. И хотя предугадать развитие событий и арки героев вроде бы пара пустяков, уверенности в том, что все будет хорошо, почему-то нет, и актеры заставляют поволноваться за своих персонажей до самого финала. Картина отлично снята и поставлена, музыкальные номера сделаны остроумно и с размахом — так, чтобы душа сначала развернулась, потом свернулась, а потом опять развернулась. То же можно сказать и о ленте в целом. Один дуэт Александра Петрова и Марии Ароновой с песней «Я свободен» в иркутском дворе уже стоит того, чтобы это увидеть и пересматривать. Но главное — картина убедительно говорит о том, что спорт — это не про победы и даже не про соревнования. А про отношения. Почему можно не смотреть Отец с гиперопекой не дает дочери воплотить мечту и не доверяет ее парню; влюбленным не суждено быть вместе, потому что ему место в Монреале, а ей в Иркутске; сибирские мужики не доверяют московскому выскочке, а он задирает перед ними нос — перевоспитаться придется всем. В принципе, такое можно снимать хоть каждый год непрерывно — если это работает, в чем проблема? Тем не менее, некоторые любят погорячее, даже если речь про лед. Музыкальные номера хороши, но песни уж больно заезжены, а Горину и Ирине Сергеевне надо обязательно напиться, чтобы запеть. Уже, честно говоря, хочется оставить алкогольный юмор в советском и постсоветском прошлом, и Крыжовников явно поставил в этом мощную точку своим «Самым лучшим днем». И уж если в саундтрек фильма «Лед» выбрана песня группы Cream Soda, чей основатель Дмитрий Свиргунов погиб, провалившись под лед, это стоило сделать как-то аккуратнее и хотя бы со смыслом.'
topic = 'Кино и телевидение'

find_similars(line, topic)

[('Фильм', 0.41618800097805864)]